In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

# Function to create the style transfer model
def create_model():
    # Define your model architecture (replace this with your model)
    model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)
    return model

# Function to define content loss
def content_loss(content_features, generated_features):
    return tf.reduce_mean(tf.square(content_features - generated_features))

# Function to define style loss using Gram matrix
def gram_matrix(x):
    return tf.linalg.einsum('bijc,bijd->bcd', x, x)

def style_loss(style_features, generated_features):
    return tf.reduce_mean(tf.square(gram_matrix(style_features) - gram_matrix(generated_features)))

# Function to calculate total loss
def total_loss(content_features, style_features, generated_features):
    alpha = 1.0  # Content weight
    beta = 0.1   # Style weight
    return alpha * content_loss(content_features, generated_features) + beta * style_loss(style_features, generated_features)

# Function to perform style transfer
def style_transfer(content_image, style_image, num_iterations=100):
    # Load and preprocess images
    content_image = tf.keras.preprocessing.image.load_img(content_image, target_size=(256, 256))
    content_image = tf.keras.preprocessing.image.img_to_array(content_image)
    content_image = tf.expand_dims(content_image, axis=0)

    style_image = tf.keras.preprocessing.image.load_img(style_image, target_size=(256, 256))
    style_image = tf.keras.preprocessing.image.img_to_array(style_image)
    style_image = tf.expand_dims(style_image, axis=0)

    # Create the style transfer model
    model = create_model()

    # Extract features from intermediate layers for content and style images
    content_features = model(content_image)
    style_features = model(style_image)

    # Initialize generated image as the content image
    generated_image = tf.Variable(content_image, trainable=True)

    # Use Adam optimizer for gradient descent
    optimizer = optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)

    # Perform gradient descent to minimize the total loss
    for i in range(num_iterations):
        with tf.GradientTape() as tape:
            tape.watch(generated_image)
            gen_features = model(generated_image)
            loss = total_loss(content_features, style_features, gen_features)

        gradients = tape.gradient(loss, generated_image)
        optimizer.apply_gradients([(gradients, generated_image)])

        # Clip values to be in the valid image range
        generated_image.assign(tf.clip_by_value(generated_image, clip_value_min=0.0, clip_value_max=255.0))

        if i % 10 == 0:
            print(f"Iteration {i}, Loss: {loss.numpy()}")

    # Display the stylized image
    plt.imshow(generated_image.numpy()[0].astype("uint8"))
    plt.axis('off')
    plt.show()

# Example usage
content_image_path = 'path/to/content_image.jpg'
style_image_path = 'path/to/style_image.jpg'
style_transfer(content_image_path, style_image_path)
